In [1]:
import collections
# Prepare paths to local utilities
import os
import sys

import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

In [2]:
is_kaggle = (os.environ.get("PWD", "") == "/kaggle/working")
print(f"Are we running in Kaggle? {is_kaggle}")

Are we running in Kaggle? False


In [3]:
if not is_kaggle:
    models_path = os.path.abspath(os.path.join('..', 'model'))
    utils_path = os.path.abspath(os.path.join('..', 'util'))
    sys.path.append(models_path)
    sys.path.append(utils_path)

    %load_ext autoreload
    %autoreload 2

    from download.DataDownloader import DataDownloader
    from collect.DataframeCollector import DataframeCollector
    from collect.TestSetSplitter import TestSetSplitter
    from collect.DatasetPreparation import DatasetPreparation
    from processing.DataPreprocessor import DataPreprocessor
    from reservoir.BasicESNCuda import BasicESNCuda as BasicESN
    from reservoir.ESNUtil import generate_input_weights

else:
    from datadownloader.datadownloader import DataDownloader
    from dataframecollector.dataframecollector import DataframeCollector
    from testsetsplitter.testsetsplitter import TestSetSplitter
    from datasetpreparation.datasetpreparation import DatasetPreparation
    from datapreprocessor.datapreprocessor import DataPreprocessor
    from basicesncuda.basicesncuda import BasicESNCuda as BasicESN
    from esnutil.esnutil import generate_input_weights

In [4]:
import joblib
from joblib import Parallel, delayed
from tqdm import tqdm
import contextlib


@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    '''
    Context manager to patch joblib to report into tqdm progress bar given as argument
    :param tqdm_object: The tqdm progress bar
    '''

    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):

        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback

    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [ ]:
data_preparation = DatasetPreparation()

input_features = ['back_x', 'back_y', 'back_z', 'thigh_x', 'thigh_y', 'thigh_z']
output_features = ['label']

data_preparation.prepare_dataset('har70plus', input_features, output_features)

Dataset already downloaded
Discovered  18  csv files in  E:\PyCharm\COM6906-Dissertation\data\har70plus
Loading the csv files into dataframes
Loaded  18  dataframes
Concatenating the dataframes
Data shape:  [(103860, 9), (131367, 9), (116413, 9), (150758, 9), (87006, 9), (122714, 9), (120125, 9), (130494, 9), (121763, 9), (122061, 9), (128063, 9), (119310, 9), (123599, 9), (101510, 9), (153517, 9), (138278, 9), (147045, 9), (141714, 9)]
Number of frames in training set: 17
Number of frames in validation set: 17
Number of frames in testing set: 18
X_train shape: (1357646, 6), Y_train shape: (1357646, 1)
X_val shape: (339404, 6), Y_val shape: (339404, 1)
X_test shape: (562547, 6), Y_test shape: (562547, 1)
Y_train encoded shape: (1357646, 7)
Y_val encoded shape: (339404, 7)
Y_test encoded shape: (562547, 7)


In [ ]:
X_train_scaled, y_train_encoded = data_preparation.get_preprocessed_data('train')
X_val_scaled, y_val_encoded = data_preparation.get_preprocessed_data('val')
X_test_scaled, y_test_encoded = data_preparation.get_preprocessed_data('test')

In [ ]:
data_preprocessor = DataPreprocessor()

# The data preprocessor object provides access to different data preprocessing methods
# The available methods are:
# - buffered_windows(window_size, x, y)
# - exponential_moving_average(alpha, x, y)
# - fourier_smoothing(x, threshold)
# - pipeline(step_names, x, y)
# - get_available_steps()

In [ ]:
# As a test, we will start by taking the optimal hyperparameter set from the basicESN testing and see whether this needs to be adjusted once the data is preprocessed

# The optimal hyperparameters were:
n_neurons = 500
density = 0.2
leakage = 0.8
spectral_radius = 0.999
gamma = 0.999
sparsity = 0.8
input_weight_type = 'uniform'

# We will first try using each preprocessing method separately to see if any of them improve the performance, and then attempt a few pipelines to see if a combination of methods can improve the performance

In [ ]:
run_windows_optimal = True

In [ ]:
# First, we will try the buffered_windows method

if run_windows_optimal:
    window_sizes = [10, 50, 100, 500, 1000]
    
    window_scores = []
    
    for window_size in window_sizes:
        x_train_windowed, y_train_windowed = data_preprocessor.buffered_windows(window_size, X_train_scaled, y_train_encoded)
        x_val_windowed, y_val_windowed = data_preprocessor.buffered_windows(window_size, X_val_scaled, y_val_encoded)
        x_test_windowed, y_test_windowed = data_preprocessor.buffered_windows(window_size, X_test_scaled, y_test_encoded)
        
        basic_esn = BasicESN(n_neurons=n_neurons, density=density, leakage=leakage, spectral_radius=spectral_radius, gamma=gamma, sparsity=sparsity, input_weight_type=input_weight_type)
        
        basic_esn.fit(x_train_windowed, y_train_windowed, x_val=x_val_windowed, y_val=y_val_windowed)
        
        y_pred = basic_esn.forward(x_test_windowed)
        
        # Before we can score the model, we need to decode the one-hot encoded labels
        # As we are using windowed data, we first need to reshape the data from (n_samples//window_size, n_classes * window_size) to (n_samples, n_classes)
        # And then we need to pass the reshaped data through the inverse_transform of data_preparation.get_encoder()
        y_pred_reshaped = y_pred.reshape(-1, y_pred.shape[1] // window_size)
        y_test_reshaped = y_test_windowed.reshape(-1, y_test_windowed.shape[1] // window_size)
        
        y_pred_decoded = data_preparation.get_encoder().inverse_transform(y_pred_reshaped)
        y_test_decoded = data_preparation.get_encoder().inverse_transform(y_test_reshaped)
        
        # Calculate the NMRSE score
        nrmse = np.sqrt(mean_squared_error(y_test_decoded, y_pred_decoded)) / (y_test_decoded.max() - y_test_decoded.min())
        
        window_scores.append({'window_size': window_size, 'nmrse': nrmse})
        
else:
    window_scores = []

In [ ]:
print(window_scores)

In [ ]:
# Let's plot the results to see if there is a clear winner
sizes = [score['window_size'] for score in window_scores]
nmrse = [score['nmrse'] for score in window_scores]

plt.figure(figsize=(10, 7))
plt.plot(sizes, nmrse, marker='o')
plt.xlabel('Window size')
plt.ylabel('NMRSE')
plt.title('NMRSE scores for different window sizes')

plt.tight_layout()
plt.show()

In [ ]:
# Get the best window size
best_window_optimal = window_scores[np.argmin(nmrse)]['window_size']

print(f"Best Window Size: {best_window_optimal}")

In [ ]:
run_emas_optimal = True

In [ ]:
# Now we will try the exponential_moving_average method

if run_emas_optimal:
    alphas = [0.001, 0.01, 0.1, 0.5, 0.7, 0.8, 0.9, 0.95, 0.99, 0.999]
    
    ema_scores = []
    
    for alpha in alphas:
        x_train_ema, y_train_ema = data_preprocessor.exponential_moving_average(alpha, X_train_scaled, y_train_encoded)
        x_val_ema, y_val_ema = data_preprocessor.exponential_moving_average(alpha, X_val_scaled, y_val_encoded)
        x_test_ema, y_test_ema = data_preprocessor.exponential_moving_average(alpha, X_test_scaled, y_test_encoded)
        
        basic_esn = BasicESN(n_neurons=n_neurons, density=density, leakage=leakage, spectral_radius=spectral_radius, gamma=gamma, sparsity=sparsity, input_weight_type=input_weight_type)
        
        basic_esn.fit(x_train_ema, y_train_ema, x_val=x_val_ema, y_val=y_val_ema)
        
        y_pred = basic_esn.forward(x_test_ema)
        
        # Before we can score the model, we need to decode the one-hot encoded labels
        y_pred_decoded = data_preparation.get_encoder().inverse_transform(y_pred)
        y_test_decoded = data_preparation.get_encoder().inverse_transform(y_test_ema)
        
        # Calculate the NMRSE score
        nrmse = np.sqrt(mean_squared_error(y_test_decoded, y_pred_decoded)) / (y_test_decoded.max() - y_test_decoded.min())
        
        ema_scores.append({'alpha': alpha, 'nmrse': nrmse})
        
else:
    ema_scores = []

In [ ]:
# Let's plot the results to see if there is a clear winner
alphas = [score['alpha'] for score in ema_scores]
nmrse = [score['nmrse'] for score in ema_scores]

plt.figure(figsize=(10, 7))

plt.plot(alphas, nmrse, marker='o')
plt.xlabel('Alpha')
plt.ylabel('NMRSE')
plt.title('NMRSE scores for different alpha values')
plt.xscale('log')

plt.tight_layout()
plt.show()

In [ ]:
# Get the best alpha
best_alpha_optimal = ema_scores[np.argmin(nmrse)]['alpha']

print(f"Best EMA Alpha: {best_alpha_optimal}")

In [ ]:
run_fourier_optimal = True

In [ ]:
# Now we will try the fourier_smoothing method

if run_fourier_optimal:
    thresholds = [1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    
    fourier_scores = []
    
    for threshold in thresholds:
        x_train_fourier = data_preprocessor.fourier_smoothing(X_train_scaled, threshold)
        x_val_fourier = data_preprocessor.fourier_smoothing(X_val_scaled, threshold)
        x_test_fourier = data_preprocessor.fourier_smoothing(X_test_scaled, threshold)
        
        basic_esn = BasicESN(n_neurons=n_neurons, density=density, leakage=leakage, spectral_radius=spectral_radius, gamma=gamma, sparsity=sparsity, input_weight_type=input_weight_type)
        
        basic_esn.fit(x_train_fourier, y_train_encoded, x_val=x_val_fourier, y_val=y_val_encoded)
        
        y_pred = basic_esn.forward(x_test_fourier)
        
        # Before we can score the model, we need to decode the one-hot encoded labels
        y_pred_decoded = data_preparation.get_encoder().inverse_transform(y_pred)
        y_test_decoded = data_preparation.get_encoder().inverse_transform(y_test_encoded)
        
        # Calculate the NMRSE score
        nrmse = np.sqrt(mean_squared_error(y_test_decoded, y_pred_decoded)) / (y_test_decoded.max() - y_test_decoded.min())
        
        fourier_scores.append({'threshold': threshold, 'nmrse': nrmse})
        
else:
    fourier_scores = []

In [ ]:
# Let's plot the results to see if there is a clear winner
thresholds = [score['threshold'] for score in fourier_scores]
nmrse = [score['nmrse'] for score in fourier_scores]

plt.figure(figsize=(10, 7))

plt.plot(thresholds, nmrse, marker='o')
plt.xlabel('Threshold')
plt.ylabel('NMRSE')
plt.title('NMRSE scores for different threshold values')
plt.xscale('log')

plt.tight_layout()
plt.show()

In [ ]:
# Get the best threshold
best_threshold_optimal = fourier_scores[np.argmin(nmrse)]['threshold']

print(f"Best Fourier Threshold: {best_threshold_optimal}")

In [ ]:
# From our basicESN testing, we know that there are some ranges of suboptimal hyperparameters, so these can be excluded from the search space to save time

# This is constant as this performed the best, whilst still computing in a reasonable time
n_neurons = 500

density_range = [0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 0.999]
leakage_range = [0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 0.999]
spectral_radius_range = [0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 0.999]
gamma_range = [0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 0.999]
sparsity_range = [0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 0.999]

input_weight_type = 'normal'